In [ ]:
import sys
sys.path.append('../qick/qick_lib/')

from mkids import *

import numpy as np

import matplotlib.pyplot as plt
from matplotlib.ticker import StrMethodFormatter
from numpy.fft import fft, fftshift

In [ ]:
# Initialize Firmware.
soc = MkidsSoc('./mkids_4x4_v2.bit')

# Name the Chains.
soc['analysis'][0]['name']  = 'ADC224 CH0, Low Frequency Balun'
soc['analysis'][1]['name']  = 'ADC224 CH1, Low Frequency Balun'
soc['analysis'][2]['name']  = 'ADC224 CH2, High Frequency Balun'
soc['analysis'][3]['name']  = 'ADC224 CH3, High Frequency Balun'
soc['synthesis'][0]['name'] = 'DAC229 CH0, High Frequency Balun'
soc['synthesis'][1]['name'] = 'DAC229 CH2, Low Frequency Balun'
soc['synthesis'][2]['name'] = 'DAC229 CH1, High Frequency Balun'
soc['synthesis'][3]['name'] = 'DAC229 CH3, Low Frequency Balun'

# Print information.
print(soc)

In [ ]:
################################
### Processing Chain Example ###
################################
# Build processing chain.
chain = KidsChain(soc, soc['analysis'][0], soc['synthesis'][1], name = "LPF 630")

print("Built MKIDs Processing Chain")
print(" ")

print("Analysis Chain: {}".format(chain.analysis.name))
print("  * sampling frequency = {} MHz".format(chain.analysis.fs))
print("  * channel separation = {} MHz".format(chain.analysis.fc_ch))
print(" ")

print("Syntheis Chain: {}".format(chain.synthesis.name))
print("  * sampling frequency = {} MHz".format(chain.synthesis.fs))
print("  * channel separation = {} MHz".format(chain.synthesis.fc_ch))
print(" ")

In [ ]:
################################
### Processing Chain Example ###
################################
# Build processing chain.
chain = KidsChain(soc, soc['analysis'][1], soc['synthesis'][3], name = "Cable Loop-Back")

print("Built MKIDs Processing Chain")
print(" ")

print("Analysis Chain: {}".format(chain.analysis.name))
print("  * sampling frequency = {} MHz".format(chain.analysis.fs))
print("  * channel separation = {} MHz".format(chain.analysis.fc_ch))
print(" ")

print("Syntheis Chain: {}".format(chain.synthesis.name))
print("  * sampling frequency = {} MHz".format(chain.synthesis.fs))
print(" ")

In [ ]:
#######################
### Frequency Sweep ###
#######################
f,a,phi=chain.sweep(100,140,N=100,g=0.05)

plt.figure(dpi=150)
plt.plot(f,20*np.log10(a/max(a)))
plt.ylim([-60,10])
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");

In [ ]:
############################
### Wide Frequency Sweep ###
############################
fstart = 0
fend = 2000

# Number of pointes per sweep.
N = 100

# Use 80 % the available bandwidth per sweep.
fbw = 0.8*min(chain.analysis.fs,chain.synthesis.fs)

if (fend-fstart)>fbw:
    fstart = np.arange(fstart, fend, fbw)

# Total number  of points.
NT = len(fstart)*N

f_v = np.zeros(NT)
a_v = np.zeros(NT)
phi_v = np.zeros(NT)
for i,ff in enumerate(fstart):
    fend_ = ff+fbw
    if fend_ > fend:
        fend_ = fend
    print("i = {}, fstart = {} MHz, fend = {} MHz.".format(i, ff, fend_))
    
    # Sweep.
    f,a,phi=chain.sweep(ff,fend_,N=N,g=0.5)
    
    # Concat values.
    f_v[i*N:(i+1)*N] = f
    a_v[i*N:(i+1)*N] = a
    phi_v[i*N:(i+1)*N] = phi
    
plt.figure(dpi=150)
plt.plot(f_v,20*np.log10(a_v/max(a_v)))
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");
#plt.savefig('wide-vna.jpg')

#plt.figure(dpi=150)
#plt.plot(f_v,phi_v)
#plt.xlabel("Frequency [MHz]");
#plt.ylabel("Phase [rad]");

In [ ]:
################################
### Multiple Synthesis Chain ###
################################
synt = [SynthesisChain(soc, soc['synthesis'][0]), SynthesisChain(soc, soc['synthesis'][3])]

# Sampling frequency.
fs=synt[0].dict['chain']['fs']

fmix = 500
for s in synt:
    # Set mixer frequency.
    s.set_mixer_frequency(fmix)

    # All channels off.
    s.alloff()

    # Set output quantization.
    s.qout(3)

    fv = np.linspace(fmix,fmix+10,7)
    for f in fv:
        print("f = {} MHz".format(f))
        s.set_tone(f, g=0.02)

In [ ]:
#########################
### Spectrum Analyzer ###
#########################

fmix = 100
chain.set_mixer_frequency(fmix)

# Set some output tones.
fv = np.linspace(fmix,fmix+10,7)
#for f in fv:
#    print("f = {} MHz".format(f))
    #chain.synthesis.set_tone(f, g=0.2)

# Set output tone.
chain.set_tone(110.6, g=0.5, verbose=True)
    
# Data source.
chain.analysis.source("input")

# Decimation.
chain.analysis.set_decimation(1)
chain.analysis.qout(3)

# Frequency range.
fstart = 100
fend   = 120
f = np.arange(fstart, fend, chain.analysis.fc_ch)
#f = [110,111,112]
print("Spectrum")
print("fstart = {} MHz, fend = {} MHz, fc = {} MHz".format(fstart, fend, chain.analysis.fc_ch))

# Set mixer to starting point.
chain.analysis.set_mixer_frequency(-fstart)

# Frequency and amplitude vectors.
FF = []
AA = []
plt.figure(dpi=150);
for i,fck in enumerate(f):
    print("i = {}, fck = {} MHz".format(i,fck))
    
    # Transfer data.
    [xi,xq] = chain.analysis.get_bin(fck)    
    x = xi + 1j*xq
    
    # Frequency vector.
    F = (np.arange(len(x))/len(x)-0.5)*chain.analysis.fs_ch
    
    # Normalization factor.
    NF = (2**15)*len(F)

    w = np.hanning(len(x))
    xw = x*w
    YY = fftshift(fft(xw))
    YYlog = 20*np.log10(abs(YY)/NF)
    AA = np.concatenate((AA,YYlog))
    
    Fk = F+fck
    FF = np.concatenate((FF,Fk))
    plt.plot(Fk,YYlog);
plt.xlabel("Frequency [MHz]");
plt.ylabel("Amplitude [dB]");
#plt.savefig('wide-spectrum.jpg')
#plt.plot(FF,AA);

In [ ]:
###############################
### Zooming-in the spectrum ###
###############################
# Set decimation.
chain.set_decimation(3)

# Set quantization.
#chain.analysis.qout(2)

# Data source.
chain.source("input")

# Get data.
fc = 110.6
[xi,xq] = chain.analysis.get_bin(fc)
xi = xi[100:]
xq = xq[100:]
x = xi + 1j*xq

w = np.hanning(len(x))
xw = x*w
F = (np.arange(len(x))/len(x)-0.5)*chain.analysis.fs_ch*1000
Y = fftshift(fft(xw))

# Normalization factor.
NF = (2**15)*len(F)

plt.figure(1,dpi=150)
plt.plot(F,20*np.log10(abs(Y)/NF))
plt.xlabel("F [kHz]");
plt.ylabel("Amplitude [dB]");
#plt.savefig('zoom-spectrum-2.jpg')

In [ ]:
########################
### Delay estimation ###
########################
# Delta frequency per sweep.
df = 10

# Number of points per sweep.
N = int(np.ceil(df/chain.fr)) + 10
#N = 200

# Starting frequency points.
fstart = 110.5 - df/2
#fstart = 110.5 - 10*chain.fr

#df = N*chain.fr

# Frequency Sweep.
f,a,phi = chain.sweep(fstart,fstart+df,N=N, g=0.05,decimation=2,set_mixer=True, verbose=False)
df, dt = chain.phase_slope(f, phi)
 
print(" ")
print("df = {} MHz, dt = {} us".format(df, dt))

plt.figure(dpi=180)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.gca().xaxis.set_major_formatter(StrMethodFormatter('{x:,.2f}'))
plt.gca().yaxis.set_major_formatter(StrMethodFormatter('{x:,.0f}'))
#plt.savefig('phase-short.jpg')

In [ ]:
##############################
### Phase Correction by DT ###
##############################
# DT is in us.
DT = 22.7 + 0.04#+ 0.022 + 0.015# + 0.008 + 0.00096

phi_u, phi_dt = chain.phase_correction(f, phi, DT=DT)

plt.figure(dpi=150)
plt.plot(f,phi)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Original Phase");

plt.figure(dpi=150)
plt.plot(f,phi_u)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Unwrap Phase");

plt.figure(dpi=150)
plt.plot(f,phi_dt)
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Correctred phase by DT = {:.6f} us".format(DT));

In [ ]:
#################################
### Overall delay computation ###
#################################
data = chain.phase_fit(f, phi_dt, jumps=False)

m = data['fits'][0]['slope']
x = data['fits'][0]['data']['x']
y = data['fits'][0]['data']['y']
fn = data['fits'][0]['data']['fn']
    
print("Slope = = {} us".format(m/(2*np.pi)))
plt.figure(dpi=150);
plt.plot(x, y, '.', x, fn, '--k');
plt.xlabel("Frequency [MHz]");
plt.ylabel("Phase [rad]");
plt.title("Slope = {:.5} ns".format(1000*m/(2*np.pi)));
#plt.savefig('phase-slope-2.jpg')

In [ ]:
####################################
### Jump-based delay computation ###
####################################
data = chain.phase_fit(f, phi_dt)

# Sampling period (ns).
ts = 1000/chain.analysis.fs

m_avg = 0
for i in range(len(data['fits'])):
    m  = data['fits'][i]['slope']
    x  = data['fits'][i]['data']['x']
    y  = data['fits'][i]['data']['y']
    fn = data['fits'][i]['data']['fn']
    
    m_avg = m_avg + m
      
    print("Slope [{}]\t= {:.5f} ns".format(i,1000*m/(2*np.pi)))
    plt.figure(dpi=180);
    plt.plot(x, y, '.', x, fn, '--k');
    plt.xlabel('Frequency [MHz]')
    plt.ylabel('Phase [rad]')
    plt.title('Slope [{}] = {:.5} ns'.format(i,1000*m/(2*np.pi)))
    #plt.savefig('phase-slope-jump-{}.jpg'.format(i))
    
m_avg = m_avg/len(data['fits'])
print("Average Slope\t= {:.5f} ns".format(1000*m_avg/(2*np.pi)))
print(" ")
    
for i in range(len(data['jump']['value'])):
    jv = data['jump']['value'][i]
    
    print("Jump [{}]\t= {:.5f} rad, {:.5f} ns, {:5f} samples".format(i, jv, 1000*jv/(2*np.pi), 1000*jv/(2*np.pi*ts)))
    
j_avg = np.mean(data['jump']['value'])
print("Average Jump\t= {:.5f} rad, {:.5f} ns, {:.5f} samples".format(j_avg, 1000*j_avg/(2*np.pi), 1000*j_avg/(2*np.pi*ts)))

In [ ]:
# Set mixer frequency.
chain.set_mixer_frequency(100)

# Set decimation.
chain.analysis.set_decimation(100)

# Base dds frequency.
fd = 0.49
#fdds = chain.analysis.fs_ch/20 - chain.fr

# Channels.
ch0 = 4*8
freq0 = chain.analysis.ch2freq(ch0)

ch1 = 4*8+1
freq1 = chain.analysis.ch2freq(ch1)

# Set output tone.
chain.synthesis.alloff()
chain.synthesis.set_tone(freq0+fd, g=0.1,verbose=True)
#chain.synthesis.set_tone(freq0+fd, g=0.1,verbose=True)

# Mask all channels.
chain.analysis.maskall()

# Set analysis tone (dds mode).
chain.analysis.source("input")

# get_bin will configure the DDS to the specified frequency.
[xi0,xq0] = chain.analysis.get_bin(f=freq0,verbose=True)
[xi1,xq1] = chain.analysis.get_bin(f=freq1,verbose=True)

# Channel Sel block.
chsel = getattr(soc,chain.analysis.dict['chain']['chsel'])

# Get data (at the same time).
data = chain.analysis.get_data_all()

# Check only 1 transaction is enabled.
if len(chsel.dict['tran']) > 1:
    print("ERROR: more than one transaction is enabled")
    
ntran = chsel.dict['tran'][0]
#xi0 = data['samples'][ntran][0,:]
#xq0 = data['samples'][ntran][1,:]
#xi1 = data['samples'][ntran][2,:]
#xq1 = data['samples'][ntran][3,:]